# Task 1b - Traveling Salesman Problem

## Define Task

In [152]:
import numpy as np

INT_MAX = 2147483647

# Positions:
LAB = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Number of cities
N_CITIES = len(LAT) - 1 # We set first city as A

# Distance betrween cities
def distance(city1, city2):
    return np.sqrt((LAT[city1] - LAT[city2])**2 + (LON[city1] - LON[city2])**2)

# Route
def route(chromosome):
    return [LAB[0]] + [LAB[i] for i in chromosome] + [LAB[0]]

## Define fitness function

In [153]:
def score_chromosome(chromosome):
    score = 0
    score += distance(0, chromosome[0]) # From A to first city in chromosome
    for i in range(N_CITIES - 2):
        score += distance(chromosome[i+1], chromosome[i+2])
    score += distance(chromosome[-1], 0) # Return to start
    return score

def fitness_function(ga_instance, solution, solution_idx):
    return score_chromosome(solution) * -1

## Define Callbacks

In [154]:
def on_generation(ga_instance):
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("Generation: ", ga_instance.generations_completed, ". Distance: ", solution_fitness * -1)
    solution_distance = print(route(solution))

## Run genetic algorithm

In [204]:
import pygad
    
ga_instance = pygad.GA(
    num_genes=N_CITIES,
    num_generations=1000,
    num_parents_mating=6,
    fitness_func=fitness_function,
    gene_type=int,
    gene_space=np.arange(1, N_CITIES + 1),
    sol_per_pop=200,
    init_range_low=1,
    init_range_high=N_CITIES,
    on_generation=on_generation,
    mutation_type="swap",
    mutation_probability=0.8,
    crossover_type="uniform",
    crossover_probability=0.8,
    parent_selection_type="tournament",
    allow_duplicate_genes=False,
    stop_criteria="saturate_1000"
)

ga_instance.run()

Generation:  1 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  2 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  3 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  4 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  5 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  6 . Distance:  37.83784230370051
['A', 'B', 'H', 'I', 'K', 'J', 'L', 'D', 'C', 'N', 'E', 'F', 'M', 'G', 'A']
Generation:  7 . Distance:  34.8480763694926
['A', 'B', 'L', 'N', 'C', 'E', 'F', 'D', 'G', 'M', 'K', 'H', 'I', 'J', 'A']
Generation:  8 . Distance:  34.8480763694926
['A', 'B', 'L', 'N', 'C', 'E', 'F', 'D', 'G', 'M', 'K', 'H', 'I', 'J', 'A']
Generation:  9 . Distance:

## Report

### 1. 

The better route found was by following this going through this order: 

A -> J -> E -> F -> L -> D -> C -> N -> G -> M -> K -> I -> H -> B -> A 

for a total of 4040.4 km. ([calculated here](https://www.daftlogic.com/projects-google-maps-distance-calculator.htm)).

It seems so as our solution provide a solution with no overlapping trajectory, which is a good sign giving the fact that these point are on Earth.

![TSP](./img/TSP.png)


### 2.

The fitness function is simply scoring the distance between each map point in the chromosome, we also add the distance from first city to first gene and end by adding the last gene to the first city. Then, we multiply the result by -1 in order for pygad to try to find the maximum value during the running phase.


### 3.

We encoded our solution by simply using the id in which the cities are stored in the LAT and LON arrays. We just added a label to each city in order to make it more human-readable.
We chose to get rid of the first city in the chromosome as it allowed us to set the same starting point for each run of the genetic algorithm. We also did not encode the last travel (going back to the start) which is not necessary and is already taken into account in the fitness function.


### 4.

The GA algorithm can be found above. As for the different params here is a quick explanation:
- mutation: swap, we chose this method as it is simple and efficient for this problem. the model can sometimes generate a new combination that might be shorter.
- crossover: uniform, it gives the model a 50% chance to take either parent's gene. Overall, it provides better balance than n-point crossover as this method have the problem to take too much from on parent and pygad does not allow further than two-point crossover. Taking too much from one parent can lead to long unefficient path in the chromosome wich in terms will not be selected in further generation. 
- population size: 200, it made calculation fast enough and still provided a good result.
- type of selection: tournament, it provided us with a better selection of the fittest which is more efficient in this problem because the real solution might be unique.
- number of generations: 1000, it was enough to get a good result and not too much to make the calculation too long. Also, since we chose strict params, the algorithm had trouble finding a better solution after 1000 generations and was often stuck without providing better solutions.

### 5.

What makes the TSP special with GA was that there is the specific need to not repeat genes. Our first attempts without this rule made the algorithm stuck at the starting point (There is no shortest path if there is no path !). To address this issue, we added two params: `gene_space` and `allow_duplicate_genes=False`. It gives pygad the instruction to eliminate solutions that might have duplicate genes after crossover and mutation. It made the algorithm functional, but ultimately it made him reject solutions for numerous generations resulting in a longer calculation time without progress. There certainly might be other machine learning algorithm better suited to solve this problem.